In [0]:
import re

import os
from os import listdir
from os.path import isfile, join

import torch
from torch.utils.data import Dataset

from PIL import Image

import xml.etree.ElementTree as ET

In [4]:
!unzip -q "drive/My Drive/data/train_data.zip"
train_folder = 'train_data/'
print('Number of files in the train folder', len(listdir(train_folder)))

!unzip -q "drive/My Drive/data/annotation.zip"
annotation_folder = 'annotation/'
print('Number of files in the annotation folder', len(listdir(annotation_folder)))

Number of files in the train folder 267
Number of files in the annotation folder 268


In [0]:
device = torch.device("cuda:0")

In [0]:
image_pattern = re.compile(r'^kinder_(?P<num>\d*)\.jpg$')
annotation_pattern = re.compile(r'^kinder_(?P<num>\d*)\.xml$')

class KinderDetectionDataset(Dataset):
  
  def __init__(self, image_folder, annotation_folder, transform=None):
    self.image_folder = image_folder
    self.annotation_folder = annotation_folder
    self.transform = transform
    self.image_names = self.__get_file_names__(image_folder, image_pattern)
    self.annotation_names = self.__get_file_names__(annotation_folder, annotation_pattern)

    assert len(self.image_names) == len(self.annotation_names)
    assert self.image_names.keys() == self.annotation_names.keys()

    self.keys = sorted(list(self.image_names.keys()))
  
  def __len__(self):
    return len(self.keys)

  def __getitem__(self, index): 
    key = self.keys[index]

    image_name = self.image_names[key]
    annotation_name = self.annotation_names[key]

    image_path = os.path.join(self.image_folder, image_name)
    annotation_path = os.path.join(self.annotation_folder, annotation_name)

    img = Image.open(image_path)
    annotation = self.__get_annotation__(annotation_path)

    if self.transform:
      img = self.transform(img)

    return img, annotation

  def __get_annotation__(self, annotation_path):
    annotation = {}
    boxes = []
    root = ET.parse(annotation_path).getroot()
    for bndbox in root.findall('object/bndbox'):
      xmin = int(bndbox[0].text)
      ymin = int(bndbox[1].text)
      xmax = int(bndbox[2].text)
      ymax = int(bndbox[3].text)
      boxes.append((xmin, ymin, xmax, ymax))

    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    num_objs = len(boxes)

    labels = torch.ones((num_objs), dtype=torch.int64)

    annotation["boxes"] = boxes
    annotation["labels"] = labels
    return annotation

  def __get_file_names__(self, folder, file_name_pattern):
    file_names = {}
    for f in listdir(folder):
      match = file_name_pattern.match(f)
      if not match:
        continue
      
      num = match.group('num')
      if num == '':
        continue
      
      file_names[int(num)] = f

    return file_names

In [12]:
data_train = KinderDetectionDataset(train_folder, annotation_folder)

for i in range(266):
  display(data_train[i])

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3000x4000 at 0x7EFDDFE1A0F0>,
 {'boxes': tensor([[1044., 1035., 2416., 2283.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3000x4000 at 0x7EFDE0407550>,
 {'boxes': tensor([[1197., 2240., 1621., 2854.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3000x4000 at 0x7EFDDFE1A0F0>,
 {'boxes': tensor([[1.0000e+00, 1.8110e+03, 8.7800e+02, 3.1500e+03]]),
  'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4000x3000 at 0x7EFDDFE1A2E8>,
 {'boxes': tensor([[2698., 2075., 3094., 2656.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3000x4000 at 0x7EFDDFE1AEB8>,
 {'boxes': tensor([[1040., 2445., 1549., 3169.]]), 'labels': tensor([1])})

...

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3000x4000 at 0x7EFDDFE5A438>,
 {'boxes': tensor([[1760., 3302., 2169., 3926.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4000x3000 at 0x7EFDDFE1AC88>,
 {'boxes': tensor([[1531., 1156., 2431., 1831.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4000x3000 at 0x7EFDDFE1A7B8>,
 {'boxes': tensor([[1114.,  910., 1747., 1635.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3000x4000 at 0x7EFDDFE1AF28>,
 {'boxes': tensor([[ 598., 2464., 1340., 3126.]]), 'labels': tensor([1])})

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4000x3000 at 0x7EFDDFE1AF28>,
 {'boxes': tensor([[1789., 1239., 2643., 2481.]]), 'labels': tensor([1])})